# Projet 7 : Implémentez un modèle de scoring

# Notebook de la partie modélisation (du prétraitement à la prédiction)

# SALMA CHAFAI

**Mission:**

* Vous êtes Data Scientist au sein d'une société financière, nommée "Prêt à dépenser", qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.
    
* L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).

* De plus, les chargés de relation client ont fait remonter le fait que les clients sont de plus en plus demandeurs de transparence vis-à-vis des décisions d’octroi de crédit. Cette demande de transparence des clients va tout à fait dans le sens des valeurs que l’entreprise veut incarner.

* "Prêt à dépenser" décide donc de développer un dashboard interactif pour que les chargés de relation client puissent à la fois expliquer de façon la plus transparente possible les décisions d’octroi de crédit, mais également permettre à leurs clients de disposer de leurs informations personnelles et de les explorer facilement. 

**Résumé de notre mission:**

 * 1- Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.

 * 2- Construire un dashboard interactif à destination des gestionnaires de la relation client permettant d'interpréter les prédictions faites par le modèle, et d’améliorer la connaissance client des chargés de relation client.

## A) Importation des bibliothèques nécessaires

### 1- Les bibliothèques usuelles et les bibliothèques de visualisation

In [1]:
 # ça nous permet d'importer numpy avec son nom np et matplotlib.pyplot as plt
%pylab inline 

# data
import pandas as pd
import numpy as np
import scipy 

# visualisation
import seaborn as sns
import missingno as msn
import matplotlib.pyplot as plt

# Librairie plotly pour les graphiques intéractives
import plotly.graph_objects as go
import plotly as plo
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode,iplot
from plotly.subplots import make_subplots

Populating the interactive namespace from numpy and matplotlib


### 2- Bibliothèques du ML

In [2]:
# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer

# outliers
from sklearn.ensemble import IsolationForest

# Preprocessing
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, KBinsDiscretizer, QuantileTransformer
from sklearn.compose import make_column_transformer, ColumnTransformer, make_column_selector
from category_encoders import TargetEncoder

# Feature engineering
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

import lime #LIME package
import lime.lime_tabular #the type of LIIME analysis we’ll do
import shap #SHAP package
import time #some of the routines take a while so we monitor the time
import os #needed to use Environment Variables in Domino

# Modèles

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import kernel_ridge
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

# Evaluation 
from sklearn.metrics import f1_score, r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_score,StratifiedKFold
from sklearn.model_selection import train_test_split

# pickle est un package qu'on utilise pour mettre dedans ou pour générer notre modèle pour le déployer dans une application
import pickle

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [3]:
import gc
import time
from contextlib import contextmanager
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## B) Jeu de données

### 1- Les 9 jeux de données

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:


# Le jeu de données "application_train"
app_train = pd.read_csv("C:/Users/salma/OneDrive/Bureau/Projet7/Data/Data_kaggle/application_train.csv", sep=',')

# Le jeu de données "application_test"
app_test = pd.read_csv("C:/Users/salma/OneDrive/Bureau/Projet7/Data/Data_kaggle/application_test.csv", sep=',')

# Le jeu de données "bureau"
bureau = pd.read_csv("C:/Users/salma/OneDrive/Bureau/Projet7/Data/Data_kaggle/bureau.csv", sep=',')

# Le jeu de données "bureau_balance"
bureau_balance = pd.read_csv("C:/Users/salma/OneDrive/Bureau/Projet7/Data/Data_kaggle/bureau_balance.csv", sep=',')

# Le jeu de données "credit_card_balance"
credit_balance = pd.read_csv("C:/Users/salma/OneDrive/Bureau/Projet7/Data/Data_kaggle/credit_card_balance.csv", sep=',')

# Le jeu de données "HomeCredit_columns_description"
col_description = pd.read_csv("C:/Users/salma/OneDrive/Bureau/Projet7/Data/Data_kaggle/HomeCredit_columns_description_encode.csv", sep=',', encoding="utf8")

# Le jeu de données "installments_payments"
installments_pay = pd.read_csv("C:/Users/salma/OneDrive/Bureau/Projet7/Data/Data_kaggle/installments_payments.csv", sep=',')

# Le jeu de données "POS_CASH_balance"
cash_balance = pd.read_csv("C:/Users/salma/OneDrive/Bureau/Projet7/Data/Data_kaggle/POS_CASH_balance.csv", sep=',')

# Le jeu de données "previous_application"
previous_app = pd.read_csv("C:/Users/salma/OneDrive/Bureau/Projet7/Data/Data_kaggle/previous_application.csv", sep=',')

In [6]:
# Un dictionnaire qui contients le nom de chaque table et les jeux de données associées
dict_data = {'Data application_train' : app_train,
             'Data application_test' : app_test,
             'Data bureau' : bureau,
             'Data bureau_balance' : bureau_balance, 
             'Data credit_card_balance' : credit_balance,
             'Data HomeCredit_columns_description' : col_description,
             'Data installments_payments' : installments_pay, 
             'Data POS_CASH_balance' : cash_balance, 
             'Data previous_application' : previous_app}

In [7]:
# Taille de chaque jeu de données 
for name, data in dict_data.items() :
    print(f"La taille du jeu de données {name} : {data.shape}")

La taille du jeu de données Data application_train : (307511, 122)
La taille du jeu de données Data application_test : (48744, 121)
La taille du jeu de données Data bureau : (1716428, 17)
La taille du jeu de données Data bureau_balance : (27299925, 3)
La taille du jeu de données Data credit_card_balance : (3840312, 23)
La taille du jeu de données Data HomeCredit_columns_description : (219, 5)
La taille du jeu de données Data installments_payments : (13605401, 8)
La taille du jeu de données Data POS_CASH_balance : (10001358, 8)
La taille du jeu de données Data previous_application : (1670214, 37)


In [8]:
# La table des descriptions des différentes colonnes des jeux de données
col_description.head(122)

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
5,8,application_{train|test}.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,NaN
6,9,application_{train|test}.csv,CNT_CHILDREN,Number of children the client has,NaN
7,10,application_{train|test}.csv,AMT_INCOME_TOTAL,Income of the client,NaN
8,11,application_{train|test}.csv,AMT_CREDIT,Credit amount of the loan,NaN
9,12,application_{train|test}.csv,AMT_ANNUITY,Loan annuity,NaN


## C) Exploration et nettoyage

On va s'inspirer du kernel fourni sur Kaggle : https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features/script pour la partie feature engineering

### 1- Fonctions utilisées

In [9]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [10]:
def verif_nan(data):
    # On affiche le taux de valeurs manquantes pour chaque colonne
    resultat = pd.DataFrame(((data.isna().sum()/data.shape[0])*100).sort_values(ascending=False))
    resultat.rename(columns={0:'Valeurs manquantes(%)'}, inplace=True)
    return resultat

In [11]:
def suppression_nan(data, seuil):
    """Cette fonction supprime les colonnes qui ont un taux de valeurs manquantes supérieurs à un seuil donné"""
    taux_nan = (data.isna().sum()/data.shape[0])*100
    data_f = data[data.columns[taux_nan < seuil]]
    return data_f

In [12]:
def impute_num(data, colonnes):
    for col in colonnes :
        data[col].fillna(data[col].median(), inplace = True)
    return data

In [13]:
def impute_cat(data, colonnes):
    for col in colonnes :
        data[col].fillna(data[col].mode()[0], inplace=True)
    return data

In [14]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

### 2- Le jeu de données app_train

#### (a) Exploration

In [15]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [17]:
col_desc_app = col_description[col_description["Table"] == "application_{train|test}.csv"]
col_desc_app.head()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


In [18]:
col_desc_app[col_desc_app['Row'] == 'DAYS_BIRTH']

,Unnamed: 0,Table,Row,Description,Special
17,20,application_{train|test}.csv,DAYS_BIRTH,Client's age in days at the time of application,time only relative to the application


In [21]:
col_desc_app[col_desc_app['Row'] == 'DAYS_EMPLOYED']['Description'].iloc[0]

'How many days before the application the person started current employment'

In [16]:
# On commence par vérifier le taux des valeurs manquantes
res = verif_nan(app_train)
res.head(50)

,Valeurs manquantes(%)
COMMONAREA_MEDI,69.872297
COMMONAREA_AVG,69.872297
COMMONAREA_MODE,69.872297
NONLIVINGAPARTMENTS_MODE,69.432963
NONLIVINGAPARTMENTS_AVG,69.432963
NONLIVINGAPARTMENTS_MEDI,69.432963
FONDKAPREMONT_MODE,68.386172
LIVINGAPARTMENTS_MODE,68.354953
LIVINGAPARTMENTS_AVG,68.354953
LIVINGAPARTMENTS_MEDI,68.354953


In [17]:
test = suppression_nan(app_train, 60)
test.shape

(307511, 105)

In [18]:
# On commence par vérifier le taux des valeurs manquantes
res = verif_nan(test)
res.head(50)

,Valeurs manquantes(%)
LANDAREA_AVG,59.376738
LANDAREA_MODE,59.376738
LANDAREA_MEDI,59.376738
BASEMENTAREA_AVG,58.515956
BASEMENTAREA_MODE,58.515956
BASEMENTAREA_MEDI,58.515956
EXT_SOURCE_1,56.381073
NONLIVINGAREA_AVG,55.179164
NONLIVINGAREA_MEDI,55.179164
NONLIVINGAREA_MODE,55.179164


In [19]:
# Les valeurs manquantes
#plt.figure(figsize = (15,10))
#sns.heatmap(test.isna())

In [20]:
#px.box(app_train, x="DAYS_BIRTH") 

In [21]:
#px.box(app_train, x="DAYS_EMPLOYED")

In [22]:
app_train['CODE_GENDER'].unique()

array(['M', 'F', 'XNA'], dtype=object)

#### (b) Nettoyage

In [23]:
# Preprocess application_train.csv and application_test.csv
def application_train(df, num_rows = None, nan_as_category = False):
    """Cette fonction nettoie"""

    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    

    
    
   # outliers 

    

    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    
    # Suppression des colonnes vides
    df = suppression_nan(df, 60)
    
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    
    # On remplace les valeurs inf par np.nan
    df.replace([np.inf, -np.inf], np.nan, inplace= True)
    
    # Les variables numériques 
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    var_num  = pd.DataFrame(df.select_dtypes(include=numerics)).columns.tolist()

    # Les variables catégoriques
    cat = ['object','bool']
    var_cat = pd.DataFrame(df.select_dtypes(include=cat)).columns.tolist()
    
    # Imputation des variables numériques et catégorielles
    df = impute_num(df, var_num)
    df = impute_cat(df, var_cat)
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    gc.collect()
    return df 

In [24]:
app_train_mod = application_train(app_train)
app_train_mod.head()


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,LANDAREA_AVG,LIVINGAREA_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,LANDAREA_MODE,LIVINGAREA_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,LANDAREA_MEDI,LIVINGAREA_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGA

In [25]:
# On revérifie le taux de valeurs manquantes
verif_nan(app_train_mod)

,Valeurs manquantes(%)
SK_ID_CURR,0.0
ORGANIZATION_TYPE_Housing,0.0
OCCUPATION_TYPE_Private service staff,0.0
OCCUPATION_TYPE_Realty agents,0.0
OCCUPATION_TYPE_Sales staff,0.0
OCCUPATION_TYPE_Secretaries,0.0
OCCUPATION_TYPE_Security staff,0.0
OCCUPATION_TYPE_Waiters/barmen staff,0.0
WEEKDAY_APPR_PROCESS_START_FRIDAY,0.0
WEEKDAY_APPR_PROCESS_START_MONDAY,0.0


### 3- Les tables bureau et bureau_balance

#### (a) Exploration

In [26]:
col_description[col_description["Table"] == "bureau.csv"]

,Unnamed: 0,Table,Row,Description,Special
122,125,bureau.csv,SK_ID_CURR,ID of loan in our sample - one loan in our sam...,hashed
123,126,bureau.csv,SK_BUREAU_ID,Recoded ID of previous Credit Bureau credit re...,hashed
124,127,bureau.csv,CREDIT_ACTIVE,Status of the Credit Bureau (CB) reported credits,NaN
125,128,bureau.csv,CREDIT_CURRENCY,Recoded currency of the Credit Bureau credit,recoded
126,129,bureau.csv,DAYS_CREDIT,How many days before current application did c...,time only relative to the application
127,130,bureau.csv,CREDIT_DAY_OVERDUE,Number of days past due on CB credit at the ti...,NaN
128,131,bureau.csv,DAYS_CREDIT_ENDDATE,Remaining duration of CB credit (in days) at t...,time only relative to the application
129,132,bureau.csv,DAYS_ENDDATE_FACT,Days since CB credit ended at the time of appl...,time only relative to the application
130,133,bureau.csv,AMT_CREDIT_MAX_OVERDUE,Maximal amount overdue on the Credit Bureau cr...,NaN
131,134,bureau.csv,CNT_CREDIT_PROLONG,How many times was the Credit Bureau credit pr...,NaN


In [27]:
bureau.dtypes

SK_ID_CURR                  int64
SK_ID_BUREAU                int64
CREDIT_ACTIVE              object
CREDIT_CURRENCY            object
DAYS_CREDIT                 int64
CREDIT_DAY_OVERDUE          int64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG          int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE                object
DAYS_CREDIT_UPDATE          int64
AMT_ANNUITY               float64
dtype: object

In [28]:
# On commence par vérifier le taux des valeurs manquantes
res = verif_nan(bureau)
res.head(50)

,Valeurs manquantes(%)
AMT_ANNUITY,71.473490
AMT_CREDIT_MAX_OVERDUE,65.513264
DAYS_ENDDATE_FACT,36.916958
AMT_CREDIT_SUM_LIMIT,34.477415
AMT_CREDIT_SUM_DEBT,15.011932
DAYS_CREDIT_ENDDATE,6.149573
AMT_CREDIT_SUM,0.000757
CREDIT_ACTIVE,0.000000
CREDIT_CURRENCY,0.000000
DAYS_CREDIT,0.000000


* On a que des variables numériques à imputer 

In [29]:
# On commence par vérifier le taux des valeurs manquantes
res = verif_nan(bureau_balance)
res.head(50)

,Valeurs manquantes(%)
SK_ID_BUREAU,0.0
MONTHS_BALANCE,0.0
STATUS,0.0


#### (b) Nettoyage

In [30]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(bureau, bb, num_rows = None, nan_as_category = True):
    

    
    
    

    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    
    # suppression des colonnes avec un taux de remplissages supérieur à 60%
    bureau_agg = suppression_nan(bureau_agg, 60)
    
    # On remplace les valeurs inf par np.nan
    bureau_agg.replace([np.inf, -np.inf], np.nan, inplace= True)
    
    # On impute les variables numériques manquantes de la table bureau
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    var_num  = pd.DataFrame(bureau_agg.select_dtypes(include=numerics)).columns.tolist()
    bureau_agg = impute_num(bureau_agg, var_num)
    
    
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

In [31]:
bureau_mod = bureau_and_balance(bureau, bureau_balance)
bureau_mod.head()

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,BURO_AMT_CREDIT_MAX_OVERDUE_MEAN,BURO_AMT_CREDIT_SUM_MAX,BURO_AMT_CREDIT_SUM_MEAN,BURO_AMT_CREDIT_SUM_SUM,BURO_AMT_CREDIT_SUM_DEBT_MAX,BURO_AMT_CREDIT_SUM_DEBT_MEAN,BURO_AMT_CREDIT_SUM_DEBT_SUM,BURO_AMT_CREDIT_SUM_OVERDUE_MEAN,BURO_AMT_CREDIT_SUM_LIMIT_MEAN,BURO_AMT_CREDIT_SUM_LIMIT_SUM,BURO_CNT_CREDIT_PROLONG_SUM,BURO_MONTHS_BALANCE_MIN_MIN,BURO_MONTHS_BALANCE_MAX_MAX,BURO_MONTHS_BALANCE_SIZE_MEAN,BURO_MONTHS_BALANCE_SIZE_SUM,BURO_CREDIT_ACTIVE_Active_MEAN,BURO_CREDIT_ACTIVE_Bad debt_MEAN,BURO_CREDIT_ACTIVE_Closed_MEAN,BURO_CREDIT_ACTIVE_Sold_MEAN,BURO_CREDIT_ACTIVE_nan_MEAN,BURO_CREDIT_CURRENCY_currency 1_MEAN,BURO_CREDIT_CURRENCY_currency 2_MEAN,BURO_CREDIT_CURRENCY_currency 3_MEAN,BURO_CREDIT_CURRENCY_currency 4_MEAN,BURO_CREDIT_CURRENCY_nan_MEAN,BURO_CREDIT_TYPE_Another type of loan_MEAN,BURO_CREDIT_TYPE_Car loan_MEAN,BURO_CREDIT_TYPE_Cash loan (non-earmarked)_MEAN,BURO_CREDIT_TYPE_Consumer credit_MEAN,BURO_CREDIT_TYPE_Credit card_MEAN,BURO_CREDIT_TYPE_Interbank credit_MEAN,BURO_CREDIT_TYPE_Loan for business development_MEAN,BURO_CREDIT_TYPE_Loan for purchase of shares (margin lending)_MEAN,BURO_CREDIT_TYPE_Loan for the purchase of equipment_MEAN,BURO_CREDIT_TYPE_Loan for working capital replenishment_MEAN,BURO_CREDIT_TYPE_Microloan_MEAN,BURO_CREDIT_TYPE_Mobile operator loan_MEAN,BURO_CREDIT_TYPE_Mortgage_MEAN,BURO_CREDIT_TYPE_Real estate loan_MEAN,BURO_CREDIT_TYPE_Unknown type of loan_MEAN,BURO_CREDIT_TYPE_nan_MEAN,BURO_STATUS_0_MEAN_MEAN,BURO_STATUS_1_MEAN_MEAN,BURO_STATUS_2_MEAN_MEAN,BURO_STATUS_3_MEAN_MEAN,BURO_STATUS_4_MEAN_MEAN,BURO_STATUS_5_MEAN_MEAN,BURO_STATUS_C_MEAN_MEAN,BURO_STATUS_X_MEAN_MEAN,BURO_STATUS_nan_MEAN_MEAN,ACTIVE_DAYS_CREDIT_MIN,ACTIVE_DAYS_CREDIT_MAX,ACTIVE_DAYS_CREDIT_MEAN,ACTIVE_DAYS_CREDIT_VAR,ACTIVE_DAYS_CREDIT_ENDDATE_MIN,ACTIVE_DAYS_CREDIT_ENDDATE_MAX,ACTIVE_DAYS_CREDIT_ENDDATE_MEAN,ACTIVE_DAYS_CREDIT_UPDATE_MEAN,ACTIVE_CREDIT_DAY_OVERDUE_MAX,ACTIVE_CREDIT_DAY_OVERDUE_MEAN,ACTIVE_AMT_CREDIT_MAX_OVERDUE_MEAN,ACTIVE_AMT_CREDIT_SUM_MAX,ACTIVE_AMT_CREDIT_SUM_MEAN,ACTIVE_AMT_CREDIT_SUM_SUM,ACTIVE_AMT_CREDIT_SUM_DEBT_MAX,ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN,ACTIVE_AMT_CREDIT_SUM_DEBT_SUM,ACTIVE_AMT_CREDIT_SUM_OVERDUE_MEAN,ACTIVE_AMT_CREDIT_SUM_LIMIT_MEAN,ACTIVE_AMT_CREDIT_SUM_LIMIT_SUM,ACTIVE_CNT_CREDIT_PROLONG_SUM,ACTIVE_MONTHS_BALANCE_SIZE_SUM,CLOSED_DAYS_CREDIT_MIN,CLOSED_DAYS_CREDIT_MAX,CLOSED_DAYS_CREDIT_MEAN,CLOSED_DAYS_CREDIT_VAR,CLOSED_DAYS_CREDIT_ENDDATE_MIN,CLOSED_DAYS_CREDIT_ENDDATE_MAX,CLOSED_DAYS_CREDIT_ENDDATE_MEAN,CLOSED_DAYS_CREDIT_UPDATE_MEAN,CLOSED_CREDIT_DAY_OVERDUE_MAX,CLOSED_CREDIT_DAY_OVERDUE_MEAN,CLOSED_AMT_CREDIT_MAX_OVERDUE_MEAN,CLOSED_AMT_CREDIT_SUM_MAX,CLOSED_AMT_CREDIT_SUM_MEAN,CLOSED_AMT_CREDIT_SUM_SUM,CLOSED_AMT_CREDIT_SUM_DEBT_MAX,CLOSED_AMT_CREDIT_SUM_DEBT_MEAN,CLOSED_AMT_CREDIT_SUM_DEBT_SUM,CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,CLOSED_CNT_CREDIT_PROLONG_SUM,CLOSED_MONTHS_BALANCE_SIZE_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,240043.666667,-1329.0,1778.0,82.428571,-93.142857,0,0.0,0.000,378000.0,207623.571429,1453365.000,373239.0,85240.928571,596686.5,0.0,0.00000,0.000,0,-51.0,0.0,24.571429,172.0,0.428571,0.0,0.571429,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.336651,0.007519,0.0,0.0,0.0,0.0,0.441240,0.214590,0.0,-559.0,-49.0,-309.333333,65110.333333,411.0,1778.0,1030.333333,-10.666667,0.0,0.0,0.0,378000.0,294675.0000,884025.000,373239.0,198895.50,596686.500,0.0,0.0000,0.000,0.0,32.0,-1572.0,-857.0,-1054.250000,119594.250000,-1329.0,-179.0,-628.500000,-155.000000,0.0,0.0,0.00000,279720.0,142

In [32]:
# On revérifie le taux de valeurs manquantes
verif_nan(bureau_mod)

,Valeurs manquantes(%)
BURO_DAYS_CREDIT_MIN,0.0
BURO_DAYS_CREDIT_MAX,0.0
ACTIVE_AMT_CREDIT_SUM_DEBT_SUM,0.0
ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN,0.0
ACTIVE_AMT_CREDIT_SUM_DEBT_MAX,0.0
ACTIVE_AMT_CREDIT_SUM_SUM,0.0
ACTIVE_AMT_CREDIT_SUM_MEAN,0.0
ACTIVE_AMT_CREDIT_SUM_MAX,0.0
ACTIVE_AMT_CREDIT_MAX_OVERDUE_MEAN,0.0
ACTIVE_CREDIT_DAY_OVERDUE_MEAN,0.0


### 4- La table previous_applications

#### (a) Exploration

In [33]:
# On commence par vérifier le taux des valeurs manquantes
res = verif_nan(previous_app)
res.head(50)

,Valeurs manquantes(%)
RATE_INTEREST_PRIVILEGED,99.643698
RATE_INTEREST_PRIMARY,99.643698
AMT_DOWN_PAYMENT,53.636480
RATE_DOWN_PAYMENT,53.636480
NAME_TYPE_SUITE,49.119754
NFLAG_INSURED_ON_APPROVAL,40.298129
DAYS_TERMINATION,40.298129
DAYS_LAST_DUE,40.298129
DAYS_LAST_DUE_1ST_VERSION,40.298129
DAYS_FIRST_DUE,40.298129


In [34]:
previous_app.dtypes

SK_ID_PREV                       int64
SK_ID_CURR                       int64
NAME_CONTRACT_TYPE              object
AMT_ANNUITY                    float64
AMT_APPLICATION                float64
AMT_CREDIT                     float64
AMT_DOWN_PAYMENT               float64
AMT_GOODS_PRICE                float64
WEEKDAY_APPR_PROCESS_START      object
HOUR_APPR_PROCESS_START          int64
FLAG_LAST_APPL_PER_CONTRACT     object
NFLAG_LAST_APPL_IN_DAY           int64
RATE_DOWN_PAYMENT              float64
RATE_INTEREST_PRIMARY          float64
RATE_INTEREST_PRIVILEGED       float64
NAME_CASH_LOAN_PURPOSE          object
NAME_CONTRACT_STATUS            object
DAYS_DECISION                    int64
NAME_PAYMENT_TYPE               object
CODE_REJECT_REASON              object
NAME_TYPE_SUITE                 object
NAME_CLIENT_TYPE                object
NAME_GOODS_CATEGORY             object
NAME_PORTFOLIO                  object
NAME_PRODUCT_TYPE               object
CHANNEL_TYPE             

#### (b) Exploration

In [35]:
# Preprocess previous_applications.csv
def previous_applications(prev, num_rows = None, nan_as_category = True):
    
   
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
    # suppression des colonnes avec un taux de remplissages supérieur à 60%
    prev = suppression_nan(prev, 60)    
    
    # outliers 
    
    # On remplace les valeurs inf par np.nan
    prev.replace([np.inf, -np.inf], np.nan, inplace= True)
    
    # Les variables numériques 
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    var_num  = pd.DataFrame(prev.select_dtypes(include=numerics)).columns.tolist()

    # Les variables catégoriques
    cat = ['object','bool']
    var_cat = pd.DataFrame(prev.select_dtypes(include=cat)).columns.tolist()
    
    # Imputation des variables numériques et catégorielles
    prev = impute_num(prev, var_num)
    prev = impute_cat(prev, var_cat)
    
    # Encodage 
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)    
    
    
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    
    
    
    
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    
    # On remplace les valeurs inf par np.nan
    prev_agg.replace([np.inf, -np.inf], np.nan, inplace= True)
    

    
    # Les variables numériques 
    var_num  = pd.DataFrame(prev_agg.select_dtypes(include=numerics)).columns.tolist()
    

    # Imputation des variables numériques et catégorielles
    prev_agg = impute_num(prev_agg, prev_agg.columns.tolist())

    

    
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

In [36]:
previous_app_mod = previous_applications(previous_app)
# On revérifie le taux de valeurs manquantes
verif_nan(previous_app_mod)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

,Valeurs manquantes(%)
PREV_AMT_ANNUITY_MIN,0.0
PREV_PRODUCT_COMBINATION_Card Street_MEAN,0.0
PREV_NAME_SELLER_INDUSTRY_Furniture_MEAN,0.0
PREV_NAME_SELLER_INDUSTRY_Industry_MEAN,0.0
PREV_NAME_SELLER_INDUSTRY_Jewelry_MEAN,0.0
PREV_NAME_SELLER_INDUSTRY_MLM partners_MEAN,0.0
PREV_NAME_SELLER_INDUSTRY_Tourism_MEAN,0.0
PREV_NAME_SELLER_INDUSTRY_XNA_MEAN,0.0
PREV_NAME_SELLER_INDUSTRY_nan_MEAN,0.0
PREV_NAME_YIELD_GROUP_XNA_MEAN,0.0


### 5- La table POS_CASH_Balance

#### (a) Exploration

In [37]:
# On commence par vérifier le taux des valeurs manquantes
res = verif_nan(cash_balance)
res.head(50)

,Valeurs manquantes(%)
CNT_INSTALMENT_FUTURE,0.260835
CNT_INSTALMENT,0.260675
SK_ID_PREV,0.000000
SK_ID_CURR,0.000000
MONTHS_BALANCE,0.000000
NAME_CONTRACT_STATUS,0.000000
SK_DPD,0.000000
SK_DPD_DEF,0.000000


In [38]:
cash_balance.dtypes

SK_ID_PREV                 int64
SK_ID_CURR                 int64
MONTHS_BALANCE             int64
CNT_INSTALMENT           float64
CNT_INSTALMENT_FUTURE    float64
NAME_CONTRACT_STATUS      object
SK_DPD                     int64
SK_DPD_DEF                 int64
dtype: object

#### (b) Nettoyage

In [39]:
# Preprocess POS_CASH_balance.csv
def pos_cash(pos, num_rows = None, nan_as_category = True):
    

    

    
    
    
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    
    # suppression des colonnes avec un taux de remplissages supérieur à 60%
    pos_agg = suppression_nan(pos_agg, 60)
    
    
    # prev
    pos_agg.replace([np.inf, -np.inf], np.nan, inplace= True)
    
    # Imputation des variables
    pos_agg = impute_num(pos_agg, pos_agg.columns.tolist())
    
    
    del pos
    gc.collect()
    return pos_agg

In [40]:
cash_balance_mod = pos_cash(cash_balance)
# On revérifie le taux de valeurs manquantes
verif_nan(cash_balance_mod)

,Valeurs manquantes(%)
POS_MONTHS_BALANCE_MAX,0.0
POS_MONTHS_BALANCE_MEAN,0.0
POS_NAME_CONTRACT_STATUS_nan_MEAN,0.0
POS_NAME_CONTRACT_STATUS_XNA_MEAN,0.0
POS_NAME_CONTRACT_STATUS_Signed_MEAN,0.0
POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,0.0
POS_NAME_CONTRACT_STATUS_Demand_MEAN,0.0
POS_NAME_CONTRACT_STATUS_Completed_MEAN,0.0
POS_NAME_CONTRACT_STATUS_Canceled_MEAN,0.0
POS_NAME_CONTRACT_STATUS_Approved_MEAN,0.0


### 6- La table installments_payments

In [41]:
# On commence par vérifier le taux des valeurs manquantes
res = verif_nan(installments_pay)
res.head(50)

,Valeurs manquantes(%)
DAYS_ENTRY_PAYMENT,0.021352
AMT_PAYMENT,0.021352
SK_ID_PREV,0.000000
SK_ID_CURR,0.000000
NUM_INSTALMENT_VERSION,0.000000
NUM_INSTALMENT_NUMBER,0.000000
DAYS_INSTALMENT,0.000000
AMT_INSTALMENT,0.000000


In [42]:
# Preprocess installments_payments.csv
def installments_payments(ins, num_rows = None, nan_as_category = True):
    

    
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    
    # suppression des colonnes avec un taux de remplissages supérieur à 60%
    ins_agg = suppression_nan(ins_agg, 60)
    
    
    # prev
    ins_agg.replace([np.inf, -np.inf], np.nan, inplace= True)
    
    # Imputation des variables
    ins_agg = impute_num(ins_agg, ins_agg.columns.tolist())
    
    del ins
    gc.collect()
    return ins_agg

In [43]:
installments_pay_mod = installments_payments(installments_pay)
# On revérifie le taux de valeurs manquantes
verif_nan(installments_pay_mod)

,Valeurs manquantes(%)
INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,0.0
INSTAL_DPD_MAX,0.0
INSTAL_DAYS_ENTRY_PAYMENT_SUM,0.0
INSTAL_DAYS_ENTRY_PAYMENT_MEAN,0.0
INSTAL_DAYS_ENTRY_PAYMENT_MAX,0.0
INSTAL_AMT_PAYMENT_SUM,0.0
INSTAL_AMT_PAYMENT_MEAN,0.0
INSTAL_AMT_PAYMENT_MAX,0.0
INSTAL_AMT_PAYMENT_MIN,0.0
INSTAL_AMT_INSTALMENT_SUM,0.0


### 7- La table credit_card_balance

In [44]:
# On vérifie le taux de valeurs manquantes
verif_nan(credit_balance)

,Valeurs manquantes(%)
AMT_PAYMENT_CURRENT,19.998063
AMT_DRAWINGS_ATM_CURRENT,19.524872
CNT_DRAWINGS_POS_CURRENT,19.524872
AMT_DRAWINGS_OTHER_CURRENT,19.524872
AMT_DRAWINGS_POS_CURRENT,19.524872
CNT_DRAWINGS_OTHER_CURRENT,19.524872
CNT_DRAWINGS_ATM_CURRENT,19.524872
CNT_INSTALMENT_MATURE_CUM,7.948208
AMT_INST_MIN_REGULARITY,7.948208
SK_ID_PREV,0.000000


In [45]:
# Preprocess credit_card_balance.csv
def credit_card_balance(cc, num_rows = None, nan_as_category = True):
    
   # outliers 

    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    
    # suppression des colonnes avec un taux de remplissages supérieur à 60%
    cc_agg = suppression_nan(cc_agg, 60)
    
    
    # prev
    cc_agg.replace([np.inf, -np.inf], np.nan, inplace= True)
    
    # Imputation des variables
    cc_agg = impute_num(cc_agg, cc_agg.columns.tolist())
    
    
    del cc
    gc.collect()
    return cc_agg

In [46]:
credit_balance_mod = credit_card_balance(credit_balance)

In [47]:
# On revérifie le taux de valeurs manquantes
verif_nan(credit_balance_mod)

,Valeurs manquantes(%)
CC_MONTHS_BALANCE_MIN,0.0
CC_SK_DPD_DEF_MEAN,0.0
CC_SK_DPD_MAX,0.0
CC_SK_DPD_MEAN,0.0
CC_SK_DPD_SUM,0.0
CC_SK_DPD_VAR,0.0
CC_SK_DPD_DEF_MIN,0.0
CC_SK_DPD_DEF_MAX,0.0
CC_SK_DPD_DEF_SUM,0.0
CC_CNT_INSTALMENT_MATURE_CUM_VAR,0.0


## D) Jeu de données mergé

In [48]:
df1 = app_train_mod.join(bureau_mod, how='left', on='SK_ID_CURR')
df2 = df1.join(previous_app_mod, how='left', on='SK_ID_CURR')
df3 = df2.join(cash_balance_mod, how='left', on='SK_ID_CURR')
df4 = df3.join(installments_pay_mod, how='left', on='SK_ID_CURR')
df_merge = df4.join(credit_balance_mod, how='left', on='SK_ID_CURR')
df_merge.shape

(307507, 765)

In [49]:
verif_nan(df_merge)

,Valeurs manquantes(%)
CC_COUNT,71.738855
CC_AMT_PAYMENT_CURRENT_VAR,71.738855
CC_AMT_RECEIVABLE_PRINCIPAL_MIN,71.738855
CC_AMT_PAYMENT_TOTAL_CURRENT_VAR,71.738855
CC_AMT_PAYMENT_TOTAL_CURRENT_SUM,71.738855
CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN,71.738855
CC_AMT_PAYMENT_TOTAL_CURRENT_MAX,71.738855
CC_AMT_PAYMENT_TOTAL_CURRENT_MIN,71.738855
CC_AMT_PAYMENT_CURRENT_SUM,71.738855
CC_AMT_RECEIVABLE_PRINCIPAL_MEAN,71.738855


In [50]:
def nettoyage_final(df):
    
    # suppression des colonnes vides
    df_mod = suppression_nan(df, 70)
    
    # Imputation des variables numériques 
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    var_num  = pd.DataFrame(df_mod.select_dtypes(include=numerics)).columns.tolist()
    df_mod = impute_num(df_mod, var_num)
    
    return df_mod

In [51]:
df_merge_nett = nettoyage_final(df_merge)
df_merge_nett.shape


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(307507, 624)

In [52]:
verif_nan(df_merge_nett)

,Valeurs manquantes(%)
SK_ID_CURR,0.0
PREV_CODE_REJECT_REASON_VERIF_MEAN,0.0
PREV_NAME_PAYMENT_TYPE_nan_MEAN,0.0
PREV_CODE_REJECT_REASON_CLIENT_MEAN,0.0
PREV_CODE_REJECT_REASON_HC_MEAN,0.0
PREV_CODE_REJECT_REASON_LIMIT_MEAN,0.0
PREV_CODE_REJECT_REASON_SCO_MEAN,0.0
PREV_CODE_REJECT_REASON_SCOFR_MEAN,0.0
PREV_CODE_REJECT_REASON_SYSTEM_MEAN,0.0
PREV_CODE_REJECT_REASON_XAP_MEAN,0.0


In [53]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [54]:
data_f = reduce_mem_usage(df_merge_nett)

Memory usage of dataframe is 1201.49 MB



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

Memory usage after optimization is: 415.26 MB
Decreased by 65.4%


In [55]:
data_f.to_csv('C:/Users/salma/OneDrive/Bureau/Projet7/CHAFAI_SALMA_2_dossier_code_112022/data_clean_vf.csv')